In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import random

# Activation Functions Class
class ActivationFunctions:
    def sigmoid(self, x):
        x = np.clip(x, -500, 500) # Clip to prevent the overflow 
        sigmoid_calc = 1 / (1 + np.exp(-x))
        return sigmoid_calc

    def relu(self, x):
        relu_calc = np.maximum(0, x)
        return relu_calc

    def tanh(self, x):
        tanh_calc = np.tanh(x)
        return tanh_calc

#THIS HAS BEEN ADDED AS THE NEW LINEAR ACTIVATION FUNCTION
    def linear_activation(self,x):
      """Linear activation function.
    
      Args:
        x: Input value or array.
    
      Returns:
        The input value itself.
      """

      return x

In [3]:
# #New function added
# import numpy as np

# def linear_activation(x):
#   """Linear activation function.

#   Args:
#     x: Input value or array.

#   Returns:
#     The input value itself.
#   """

#   return x

In [4]:
# Layer Class
class Layer:
    '''
        One NN layer that provides forward propagation
        Args:
            input_size: The number of neurons in the previous layer (or input features).
            output_size: The number of neurons in the current layer.
            activation_function: The activation function to be used for this layer (e.g., sigmoid, relu, tanh).

    '''
    def __init__(self, input_size, output_size, activation_function):
        # initialize weights and scaled down (* 0.01)
        self.weights = np.random.randn(output_size, input_size) * 0.01  
        # Initialize biases
        self.biases = np.zeros((output_size, 1))  
        self.activation_function = activation_function

    def forward(self, input_data):
        # Linear combination: Z = W * X + b
        z = np.dot(self.weights, input_data) + self.biases
        # Apply activation function
        a = self.activation_function(z)
        return a

class Network:
    def __init__(self, number_of_inputs, number_of_nodes, activation_functions):
        self.layers = []
        input_size = number_of_inputs

        # Create each layer based on user input
        for i in range(len(number_of_nodes)):
            layer = Layer(
                input_size=input_size,
                output_size=number_of_nodes[i],
                activation_function=activation_functions[i]
            )
            self.layers.append(layer)
            input_size = number_of_nodes[i]  # Update input size for the next layer

    def forward(self, input_data):
        # Pass data through each layer
        output = input_data
        for layer in self.layers:
            output = layer.forward(output)
        return output

    def set_weights_and_biases(self, global_best_position):
        # Set weights and biases using the global_best_position from PSO
        current_index = 0
        for layer in self.layers:
            weight_shape = layer.weights.shape
            bias_shape = layer.biases.shape
            
            # Extract weights from global_best_position
            num_weights = weight_shape[0] * weight_shape[1]
            weights = global_best_position[current_index:current_index + num_weights].reshape(weight_shape)
            current_index += num_weights

            # Extract biases from global_best_position
            num_biases = bias_shape[0] * bias_shape[1]
            biases = global_best_position[current_index:current_index + num_biases].reshape(bias_shape)
            current_index += num_biases

            # Assign the extracted weights and biases to the layer
            layer.weights = weights
            layer.biases = biases

# ANNBuilder Class
class ANNBuilder:
    def __init__(self):
        pass
        
    def build(self, number_of_inputs, number_of_nodes, activation_functions):
        # Validate user input lengths
        if len(number_of_nodes) != len(activation_functions):
            raise ValueError("The number of layers in 'number_of_nodes' must match the number of 'activation_functions'.")
        
        # Create and return the network
        return Network(number_of_inputs, number_of_nodes, activation_functions)

def validate_user_inputs(number_of_inputs, number_of_nodes, activation_functions):
    """
    Validates user inputs for creating a neural network.

    Args:
        number_of_inputs: The number of input features.
        number_of_nodes: A list containing the number of neurons in each hidden layer and the output layer.
        activation_functions: A list of activation functions, one for each layer.

    Returns:
        True if the inputs are valid, False otherwise.
    """
    if len(number_of_nodes) != len(activation_functions):
        print("Error: The number of layers in 'number_of_nodes' must be equal to the number of 'activation_functions'.")
        return False

    if not isinstance(number_of_inputs, int) or number_of_inputs <= 0 or number_of_inputs == "":
        print("Error: The 'number_of_inputs' must be a positive integer.")
        return False

    for num_nodes in number_of_nodes:
        if not isinstance(num_nodes, int) or num_nodes <= 0 or number_of_nodes == "":
            print("Error: Each entry in 'number_of_nodes' must be a positive integer.")
            return False

    return True


In [5]:

from sklearn.metrics import mean_squared_error, mean_absolute_error

def mae(y_true, y_pred):
    """
    Calculates the Mean Absolute Error (MAE) between true and predicted values.

    Args:
        y_true: The true target values.
        y_pred: The predicted values from the model.

    Returns:
        The Mean Absolute Error value.
    """
    mae_value = np.mean(np.abs(y_true - y_pred))
    return mae_value

def mse_manual(y_true, y_pred):
    """
    Calculates the Mean Squared Error (MSE) between true and predicted values.

    Args:
        y_true: The true target values.
        y_pred: The predicted values from the model.

    Returns:
        The Mean Squared Error value.
    """
    return np.mean((y_true - y_pred) ** 2)

def rmse(y_true, y_pred):
    """
    Calculates the Root Mean Squared Error (RMSE) between true and predicted values.

    Args:
        y_true: The true target values.
        y_pred: The predicted values from the model.

    Returns:
        The Root Mean Squared Error value.
    """
    rmse_calc = np.sqrt(mse_manual(y_true, y_pred))
    # print(f"rmse_calc = {rmse_calc} ")
    return rmse_calc

In [6]:
# Particle Swarm Optimization (PSO) Implementation
class Particle:
    def __init__(self, position_size):
        self.position = np.random.randn(position_size) * 0.01  # Randomly initialize position (weights and biases)
        self.velocity = np.random.randn(position_size) * 0.01  # Randomly initialize velocity
        self.best_position = np.copy(self.position)  # Best position (personal best)
        self.best_fitness = float('inf')  # Set initial fitness to a large number

class PSO:
    def __init__(self, swarm_size, input_size, number_of_nodes, activation_functions, alpha, beta, gamma, delta, epsilon, max_iterations):
        # Calculate the total number of weights and biases in the network
        total_parameters = 0
        layer_input_size = input_size
        for nodes in number_of_nodes:
            total_parameters += nodes * layer_input_size  # Weights
            total_parameters += nodes  # Biases
            layer_input_size = nodes

        self.swarm_size = swarm_size
        self.particles = [Particle(total_parameters) for _ in range(swarm_size)]
        self.global_best_position = None
        self.global_best_fitness = float('inf')
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta
        self.epsilon = epsilon
        self.max_iterations = max_iterations
        self.input_size = input_size
        self.number_of_nodes = number_of_nodes
        self.activation_functions = activation_functions

    def fitness(self, particle_position):
        # Create ANN using particle_position as weights and biases
        ann = ANNBuilder().build(self.input_size, self.number_of_nodes, self.activation_functions)
        # Set the particle weights and biases to the network
        ann.set_weights_and_biases(particle_position)
        # Forward pass and calculate fitness
        input_data = X_train.T  # Transpose to match input shape
        output = ann.forward(input_data)
        return mse_manual(y_train.T, output)  # Fitness is Mean Squared Error

    def optimize(self):
        for iteration in range(self.max_iterations):
            for particle in self.particles:
                # Assess Fitness
                fitness_value = self.fitness(particle.position)
                if fitness_value < particle.best_fitness:
                    particle.best_fitness = fitness_value
                    particle.best_position = np.copy(particle.position)
                if fitness_value < self.global_best_fitness:
                    self.global_best_fitness = fitness_value
                    self.global_best_position = np.copy(particle.position)

            # Update Velocity and Position for each Particle
            for particle in self.particles:
                # Random informants selection
                informants = random.sample(self.particles, int(len(self.particles) / 2))
                informants_best = min(informants, key=lambda p: p.best_fitness).best_position

                for i in range(len(particle.position)):
                    b = random.uniform(0, self.beta)
                    c = random.uniform(0, self.gamma)
                    d = random.uniform(0, self.delta)

                    particle.velocity[i] = (
                        self.alpha * particle.velocity[i]
                        + b * (particle.best_position[i] - particle.position[i])
                        + c * (informants_best[i] - particle.position[i])
                        + d * (self.global_best_position[i] - particle.position[i])
                    )
                # Update position
                particle.position += self.epsilon * particle.velocity

            # print(f"Iteration {iteration + 1}/{self.max_iterations}, Global Best Fitness: {self.global_best_fitness}")
            # print(f"pso.global_best_position = {pso.global_best_position}")
            # print(f"pso.global_best_position.shape = {pso.global_best_position.shape}")


In [ ]:
#RUNNING THE ANN 10 TIMES AND TAKING THE AVG OF MAE

import numpy as np

# Putting It All Together
if __name__ == "__main__":
    # Load dataset
    data = pd.read_csv('concrete_data.csv')
    
    # Separate features and target
    X = data.iloc[:, :-1].values  # All rows, all columns except the last one
    y = data.iloc[:, -1].values   # All rows, only the last column (compressive strength)

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Normalize the feature values
    min_max_scaler_X = MinMaxScaler()
    X_train = min_max_scaler_X.fit_transform(X_train)
    X_test = min_max_scaler_X.transform(X_test)
    
    # Normalize the target values separately
    min_max_scaler_y = MinMaxScaler()
    y_train = min_max_scaler_y.fit_transform(y_train.reshape(-1, 1))
    y_test = min_max_scaler_y.transform(y_test.reshape(-1, 1))

    # Create an instance of ActivationFunctions
    af = ActivationFunctions()

    # User inputs for the network
    number_of_inputs = 8  # Number of input features
    number_of_nodes = [8, 5, 3, 1]  # Adding more neurons per layer
    activation_functions = [af.tanh, af.tanh, af.relu, af.sigmoid]

    # PSO parameters
    swarm_size = 50
    alpha = 0.5
    beta = 1.0
    gamma = 1.0
    delta = 1.0
    epsilon = 1.2
    max_iterations = 100

    # Lists to store MAE results
    mae_train_results = []
    mae_test_results = []

    # Run the code 10 times
    for _ in range(10):
        # Run PSO to optimize the ANN
        pso = PSO(swarm_size, number_of_inputs, number_of_nodes, activation_functions, alpha, beta, gamma, delta, epsilon, max_iterations)
        pso.optimize()

        # Create the network with optimized weights
        ann = ANNBuilder().build(number_of_inputs, number_of_nodes, activation_functions)
        ann.set_weights_and_biases(pso.global_best_position)

        # Perform a forward pass with training data
        input_data = X_train.T  # Transpose to match input shape
        output = ann.forward(input_data)

        # Inverse transform the predictions to the original scale
        output_original_scale = min_max_scaler_y.inverse_transform(output.T)

        # Calculate Mean Absolute Error on training data in original scale
        mae_train_original = mae(min_max_scaler_y.inverse_transform(y_train), output_original_scale)
        mae_train_results.append(mae_train_original)

        # Perform a forward pass with test data
        test_output = ann.forward(X_test.T)

        # Inverse transform the predictions to the original scale
        test_output_original_scale = min_max_scaler_y.inverse_transform(test_output.T)

        # Calculate Mean Absolute Error on test data in original scale
        mae_test_original = mae(min_max_scaler_y.inverse_transform(y_test), test_output_original_scale)
        mae_test_results.append(mae_test_original)

    # Calculate average and standard deviation of MAE for training and test data
    avg_mae_train = np.mean(mae_train_results)
    std_mae_train = np.std(mae_train_results)
    avg_mae_test = np.mean(mae_test_results)
    std_mae_test = np.std(mae_test_results)

    print(f"Average MAE on training data (original scale): {avg_mae_train:.2f} ± {std_mae_train:.2f}")
    print(f"Average MAE on test data (original scale): {avg_mae_test:.2f} ± {std_mae_test:.2f}")


Average MAE on training data (original scale): 7.40 ± 1.80
Average MAE on test data (original scale): 7.54 ± 1.96


In [20]:
import numpy as np

# Putting It All Together
if __name__ == "__main__":
    # Load dataset
    data = pd.read_csv('concrete_data.csv')
    
    # Separate features and target
    X = data.iloc[:, :-1].values  # All rows, all columns except the last one
    y = data.iloc[:, -1].values   # All rows, only the last column (compressive strength)

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Normalize the feature values
    min_max_scaler_X = MinMaxScaler()
    X_train = min_max_scaler_X.fit_transform(X_train)
    X_test = min_max_scaler_X.transform(X_test)
    
    # Normalize the target values separately
    min_max_scaler_y = MinMaxScaler()
    y_train = min_max_scaler_y.fit_transform(y_train.reshape(-1, 1))
    y_test = min_max_scaler_y.transform(y_test.reshape(-1, 1))

    # Create an instance of ActivationFunctions
    af = ActivationFunctions()

    # User inputs for the network
    number_of_inputs = 8  # Number of input features
    number_of_nodes = [8, 5, 3, 1]  # Adding more neurons per layer
    activation_functions = [af.tanh, af.tanh, af.relu, af.sigmoid]

    # Different PSO configurations
    configs = [
        {"swarm_size": 10, "iterations": 50},
        {"swarm_size": 50, "iterations": 10}
    ]

    # Run each configuration
    for config in configs:
        swarm_size = config["swarm_size"]
        max_iterations = config["iterations"]

        # Lists to store MAE results
        mae_train_results = []
        mae_test_results = []

        print(f"\nRunning configuration: Swarm size = {swarm_size}, Iterations = {max_iterations}")

        # Run the code 10 times for the current configuration
        for _ in range(10):
            # Run PSO to optimize the ANN
            pso = PSO(swarm_size, number_of_inputs, number_of_nodes, activation_functions, alpha=0.5, beta=1.0, gamma=1.0, delta=1.0, epsilon=1.2, max_iterations=max_iterations)
            pso.optimize()

            # Create the network with optimized weights
            ann = ANNBuilder().build(number_of_inputs, number_of_nodes, activation_functions)
            ann.set_weights_and_biases(pso.global_best_position)

            # Perform a forward pass with training data
            input_data = X_train.T  # Transpose to match input shape
            output = ann.forward(input_data)

            # Inverse transform the predictions to the original scale
            output_original_scale = min_max_scaler_y.inverse_transform(output.T)

            # Calculate Mean Absolute Error on training data in original scale
            mae_train_original = mae(min_max_scaler_y.inverse_transform(y_train), output_original_scale)
            mae_train_results.append(mae_train_original)

            # Perform a forward pass with test data
            test_output = ann.forward(X_test.T)

            # Inverse transform the predictions to the original scale
            test_output_original_scale = min_max_scaler_y.inverse_transform(test_output.T)

            # Calculate Mean Absolute Error on test data in original scale
            mae_test_original = mae(min_max_scaler_y.inverse_transform(y_test), test_output_original_scale)
            mae_test_results.append(mae_test_original)

        # Calculate average and standard deviation of MAE for training and test data
        avg_mae_train = np.mean(mae_train_results)
        std_mae_train = np.std(mae_train_results)
        avg_mae_test = np.mean(mae_test_results)
        std_mae_test = np.std(mae_test_results)

        print(f"Average MAE on training data (original scale): {avg_mae_train:.2f} ± {std_mae_train:.2f}")
        print(f"Average MAE on test data (original scale): {avg_mae_test:.2f} ± {std_mae_test:.2f}")



Running configuration: Swarm size = 10, Iterations = 50
Average MAE on training data (original scale): 13.52 ± 0.03
Average MAE on test data (original scale): 13.31 ± 0.03

Running configuration: Swarm size = 50, Iterations = 10
Average MAE on training data (original scale): 13.54 ± 0.00
Average MAE on test data (original scale): 13.33 ± 0.00


Generally, having a moderately sized swarm with enough iterations provides a good balance between exploration (diverse solutions) and exploitation (fine-tuning)

In [8]:
# import pandas as pd
# import numpy as np
# from sklearn.metrics import mean_absolute_error, mean_squared_error
# from itertools import product

# data = pd.read_csv('concrete_data.csv')
    
# # Separate features and target
# X = data.iloc[:, :-1].values  # All rows, all columns except the last one
# y = data.iloc[:, -1].values   # All rows, only the last column (compressive strength)

# # Split data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Normalize the feature values
# min_max_scaler_X = MinMaxScaler()
# X_train = min_max_scaler_X.fit_transform(X_train)
# X_test = min_max_scaler_X.transform(X_test)

# # Normalize the target values separately
# min_max_scaler_y = MinMaxScaler()
# y_train = min_max_scaler_y.fit_transform(y_train.reshape(-1, 1))
# y_test = min_max_scaler_y.transform(y_test.reshape(-1, 1))

# # Create an instance of ActivationFunctions
# af = ActivationFunctions()

# # Sample hyperparameter ranges
# ann_topologies = [
#     {'nodes': [8, 5, 3, 1], 'activations': ['tanh', 'tanh', 'relu', 'sigmoid']},
#     {'nodes': [8, 5, 3, 1], 'activations': ['tanh', 'tanh', 'relu', 'linear_activation']},
#     {'nodes': [8, 5, 3, 1], 'activations': ['tanh', 'tanh', 'relu', 'relu']},
#     # Add more topologies as needed
# ]

# pso_params = [
#     {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100},
#     {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100},
#     {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100},
    
#     # Add more PSO configurations as needed
# ]

# # Initialize an empty DataFrame to store results
# results = pd.DataFrame(columns=[
#     'ANN_Topology', 'PSO_Params', 'MAE_Train', 'MAE_Test'
# ])

# for ann_config, pso_config in product(ann_topologies, pso_params):
#     # Define the ANN architecture from the configuration
#     nodes = ann_config['nodes']
#     activations = [getattr(af, func) for func in ann_config['activations']]  # Ensure af is defined

#     # Define the ANN input layer size
#     number_of_inputs = 8  # Example input feature count

#     # Validate inputs and build the ANN
#     if validate_user_inputs(number_of_inputs, nodes, activations):
#         ann = ANNBuilder().build(number_of_inputs, nodes, activations)
        
#         # Configure PSO parameters here
#         # swarm_size = 50
#         # max_iterations = 100
#         alpha, beta, gamma, delta, epsilon, swarm_size,max_iterations = (
#             pso_config['alpha'], pso_config['beta'], pso_config['gamma'],
#             pso_config['delta'], pso_config['epsilon'],pso_config['swarm_size'],pso_config['max_iterations']
#         )

#         # Run PSO to optimize the ANN weights
#         pso = PSO(swarm_size, number_of_inputs, nodes, activations, alpha, beta, gamma, delta, epsilon, max_iterations)
#         pso.optimize()

#         # Set ANN weights and biases with PSO's best solution
#         ann.set_weights_and_biases(pso.global_best_position)

#         # Forward pass on training and test data
#         output_train = ann.forward(X_train.T)
#         output_test = ann.forward(X_test.T)

#         # Rescale predictions back to original scale
#         output_train_original = min_max_scaler_y.inverse_transform(output_train.T)
#         output_test_original = min_max_scaler_y.inverse_transform(output_test.T)

#         # Calculate error metrics
#         mae_train = mean_absolute_error(min_max_scaler_y.inverse_transform(y_train), output_train_original)
        
#         mae_test = mean_absolute_error(min_max_scaler_y.inverse_transform(y_test), output_test_original)

#         # Append results
#         # Append results
#         new_row = pd.DataFrame([{
#             'ANN_Topology': ann_config,
#             'PSO_Params': pso_config,
#             'MAE_Train': mae_train,
#             'MAE_Test': mae_test
#         }])

#         results = pd.concat([results, new_row], ignore_index=True)


# # Save results to CSV for analysis
# results.to_csv("experiment_results.csv", index=False)

# print("Experiment completed. Results saved to 'experiment_results.csv'.")


In [9]:
# import pandas as pd
# import numpy as np
# from sklearn.metrics import mean_absolute_error, mean_squared_error
# from itertools import product

# data = pd.read_csv('concrete_data.csv')
    
# # Separate features and target
# X = data.iloc[:, :-1].values  # All rows, all columns except the last one
# y = data.iloc[:, -1].values   # All rows, only the last column (compressive strength)

# # Split data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Normalize the feature values
# min_max_scaler_X = MinMaxScaler()
# X_train = min_max_scaler_X.fit_transform(X_train)
# X_test = min_max_scaler_X.transform(X_test)

# # Normalize the target values separately
# min_max_scaler_y = MinMaxScaler()
# y_train = min_max_scaler_y.fit_transform(y_train.reshape(-1, 1))
# y_test = min_max_scaler_y.transform(y_test.reshape(-1, 1))

# # Create an instance of ActivationFunctions
# af = ActivationFunctions()

# # Sample hyperparameter ranges
# ann_topologies = [
#     {'nodes': [8, 5, 3, 1], 'activations': ['tanh', 'tanh', 'relu', 'sigmoid']},
#     {'nodes': [8, 5, 3, 1], 'activations': ['tanh', 'tanh', 'relu', 'linear_activation']},
#     {'nodes': [8, 5, 3, 1], 'activations': ['tanh', 'tanh', 'relu', 'linear_activation']},
#     # Add more topologies as needed
# ]

# pso_params = [
#     {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100},
#     {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100},
#     {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500},
    
#     # Add more PSO configurations as needed
# ]

# # Initialize an empty DataFrame to store results
# results = pd.DataFrame(columns=[
#     'ANN_Topology', 'PSO_Params', 'MAE_Train', 'MAE_Test'
# ])

# for ann_config, pso_config in product(ann_topologies, pso_params):
#     # Define the ANN architecture from the configuration
#     nodes = ann_config['nodes']
#     activations = [getattr(af, func) for func in ann_config['activations']]  # Ensure af is defined

#     # Define the ANN input layer size
#     number_of_inputs = 8  # Example input feature count

#     # Validate inputs and build the ANN
#     if validate_user_inputs(number_of_inputs, nodes, activations):
#         ann = ANNBuilder().build(number_of_inputs, nodes, activations)
        
#         # Configure PSO parameters here
#         # swarm_size = 50
#         # max_iterations = 100
#         alpha, beta, gamma, delta, epsilon, swarm_size,max_iterations = (
#             pso_config['alpha'], pso_config['beta'], pso_config['gamma'],
#             pso_config['delta'], pso_config['epsilon'],pso_config['swarm_size'],pso_config['max_iterations']
#         )

#         # Run PSO to optimize the ANN weights
#         pso = PSO(swarm_size, number_of_inputs, nodes, activations, alpha, beta, gamma, delta, epsilon, max_iterations)
#         pso.optimize()

#         # Set ANN weights and biases with PSO's best solution
#         ann.set_weights_and_biases(pso.global_best_position)

#         # Forward pass on training and test data
#         output_train = ann.forward(X_train.T)
#         output_test = ann.forward(X_test.T)

#         # Rescale predictions back to original scale
#         output_train_original = min_max_scaler_y.inverse_transform(output_train.T)
#         output_test_original = min_max_scaler_y.inverse_transform(output_test.T)

#         # Calculate error metrics
#         mae_train = mean_absolute_error(min_max_scaler_y.inverse_transform(y_train), output_train_original)
        
#         mae_test = mean_absolute_error(min_max_scaler_y.inverse_transform(y_test), output_test_original)

#         # Append results
#         # Append results
#         new_row = pd.DataFrame([{
#             'ANN_Topology': ann_config,
#             'PSO_Params': pso_config,
#             'MAE_Train': mae_train,
#             'MAE_Test': mae_test
#         }])

#         results = pd.concat([results, new_row], ignore_index=True)


# # Save results to CSV for analysis
# results.to_csv("experiment_results.csv", index=False)

# print("Experiment completed. Results saved to 'experiment_results.csv'.")


In [ ]:
#FINDING ALL POSSIBLE COMBINATIONS OF PSO PARAMETERS

from itertools import product

# Define parameter ranges for each PSO parameter
alpha_values = [0.5, 1.0]
beta_values = [0.5, 1.0, 1.5]
gamma_values = [0.5, 1.0, 1.5]
delta_values = [0.5, 1]
epsilon_values = [0.8, 1.0, 1.2]
swarm_size_values = [50, 100, 200]
max_iterations_values = [50, 100, 500]

# Generate all combinations of parameters
param_combinations = [
    {
        'alpha': alpha,
        'beta': beta,
        'gamma': gamma,
        'delta': delta,
        'epsilon': epsilon,
        'swarm_size': swarm_size,
        'max_iterations': max_iterations
    }
    for alpha, beta, gamma, delta, epsilon, swarm_size, max_iterations in product(
        alpha_values,
        beta_values,
        gamma_values,
        delta_values,
        epsilon_values,
        swarm_size_values,
        max_iterations_values
    )
]

# Display the list in the desired format
for config in param_combinations:
    print(config,',')


{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm

In [17]:

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Assuming necessary classes and functions (like ActivationFunctions, ANNBuilder, PSO, etc.) are defined
# Load the dataset
data = pd.read_csv('concrete_data.csv')

# Separate features and target
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize the feature values
min_max_scaler_X = MinMaxScaler()
X_train = min_max_scaler_X.fit_transform(X_train)
X_test = min_max_scaler_X.transform(X_test)

# Normalize the target values separately
min_max_scaler_y = MinMaxScaler()
y_train = min_max_scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test = min_max_scaler_y.transform(y_test.reshape(-1, 1))

# Create an instance of ActivationFunctions
af = ActivationFunctions()

# Define ANN and PSO configurations
ann_topologies = [
    {'nodes': [8, 10, 5, 1], 'activations': ['tanh', 'relu', 'sigmoid', 'linear_activation']},
]

pso_params = [
#All possible Different parameters
##{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100},

{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 0.5, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 0.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.0, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 0.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 0.5, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 0.8, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.0, 'swarm_size': 200, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 50, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 100, 'max_iterations': 500} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 50} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 100} ,
{'alpha': 1.0, 'beta': 1.5, 'gamma': 1.5, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500} ,
]



# Initialize an empty DataFrame to store results
results = pd.DataFrame(columns=['ANN_Topology', 'PSO_Params', 'MAE_Train', 'MAE_Test'])

# Iterate over each ANN topology and PSO configuration
for ann_config in ann_topologies:
    nodes = ann_config['nodes']
    activations = [getattr(af, func) for func in ann_config['activations']]
    number_of_inputs = 8  # Input feature count

    for pso_config in pso_params:
        if validate_user_inputs(number_of_inputs, nodes, activations):
            ann = ANNBuilder().build(number_of_inputs, nodes, activations)
            
            # Extract PSO parameters
            alpha, beta, gamma, delta, epsilon, swarm_size, max_iterations = (
                pso_config['alpha'], pso_config['beta'], pso_config['gamma'],
                pso_config['delta'], pso_config['epsilon'], pso_config['swarm_size'], pso_config['max_iterations']
            )

            # Run PSO to optimize the ANN weights
            pso = PSO(swarm_size, number_of_inputs, nodes, activations, alpha, beta, gamma, delta, epsilon, max_iterations)
            pso.optimize()

            # Set ANN weights and biases with PSO's best solution
            ann.set_weights_and_biases(pso.global_best_position)

            # Forward pass on training and test data
            output_train = ann.forward(X_train.T)
            output_test = ann.forward(X_test.T)

            # Rescale predictions back to original scale
            output_train_original = min_max_scaler_y.inverse_transform(output_train.T)
            output_test_original = min_max_scaler_y.inverse_transform(output_test.T)

            # Calculate MAE for training and test sets
            mae_train = mean_absolute_error(min_max_scaler_y.inverse_transform(y_train), output_train_original)
            mae_test = mean_absolute_error(min_max_scaler_y.inverse_transform(y_test), output_test_original)

                        # Replace the append method with pd.concat
            results = pd.concat([results, pd.DataFrame([{
                'ANN_Topology': ann_config,
                'PSO_Params': pso_config,
                'MAE_Train': mae_train,
                'MAE_Test': mae_test
            }])], ignore_index=True)

            # Debug: Print MAE values for each combination
            print(f"ANN Config: {ann_config}")
            print(f"PSO Config: {pso_config}")
            print(f"MAE Train: {mae_train}")
            print(f"MAE Test: {mae_test}")
            print("-" * 40)

# Save results to CSV for analysis
results.to_csv("experiment_results.csv", index=False)

print("Experiment completed. Results saved to 'experiment_results.csv'.")


C:\Users\Hp\AppData\Local\Temp\ipykernel_1420\381904459.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([{


ANN Config: {'nodes': [8, 10, 5, 1], 'activations': ['tanh', 'relu', 'sigmoid', 'linear_activation']}
PSO Config: {'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 50}
MAE Train: 24.49418027089773
MAE Test: 23.420836093611932
----------------------------------------
ANN Config: {'nodes': [8, 10, 5, 1], 'activations': ['tanh', 'relu', 'sigmoid', 'linear_activation']}
PSO Config: {'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 100}
MAE Train: 26.620148594578566
MAE Test: 25.531734765858015
----------------------------------------
ANN Config: {'nodes': [8, 10, 5, 1], 'activations': ['tanh', 'relu', 'sigmoid', 'linear_activation']}
PSO Config: {'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'delta': 0.5, 'epsilon': 0.8, 'swarm_size': 50, 'max_iterations': 500}
MAE Train: 25.29057119307303
MAE Test: 24.20287268786813
----------------------------------------
ANN Config: {'nodes': [

KeyboardInterrupt: 

In [18]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Assuming necessary classes and functions (like ActivationFunctions, ANNBuilder, PSO, etc.) are defined
# Load the dataset
data = pd.read_csv('concrete_data.csv')

# Separate features and target
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize the feature values
min_max_scaler_X = MinMaxScaler()
X_train = min_max_scaler_X.fit_transform(X_train)
X_test = min_max_scaler_X.transform(X_test)

# Normalize the target values separately
min_max_scaler_y = MinMaxScaler()
y_train = min_max_scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test = min_max_scaler_y.transform(y_test.reshape(-1, 1))

# Create an instance of ActivationFunctions
af = ActivationFunctions()

# Define ANN and PSO configurations
ann_topologies = [
    # {'nodes': [8, 10, 5, 1], 'activations': ['sigmoid', 'relu', 'tanh', 'linear_activation']},
    # {'nodes': [8, 5, 3, 1], 'activations': ['tanh', 'tanh', 'relu', 'linear_activation']},
    # {'nodes': [8, 6, 4, 1], 'activations': ['relu', 'tanh', 'tanh', 'linear_activation']},
    # {'nodes': [8, 10, 5, 1], 'activations': ['sigmoid', 'relu', 'tanh', 'linear_activation']},
    # {'nodes': [8, 3, 2, 1], 'activations': ['relu', 'relu', 'sigmoid', 'linear_activation']},
    # {'nodes': [8, 7, 5, 1], 'activations': ['tanh', 'sigmoid', 'relu', 'linear_activation']},
    # {'nodes': [8, 4, 4, 1], 'activations': ['relu', 'sigmoid', 'sigmoid', 'linear_activation']},
    # {'nodes': [8, 6, 2, 1], 'activations': ['tanh', 'relu', 'tanh', 'linear_activation']},
    # {'nodes': [8, 5, 3, 1], 'activations': ['sigmoid', 'sigmoid', 'relu', 'linear_activation']},
    # {'nodes': [8, 8, 4, 1], 'activations': ['relu', 'tanh', 'sigmoid', 'linear_activation']},
    {'nodes': [8, 9, 6, 1], 'activations': ['tanh', 'relu', 'sigmoid', 'linear_activation']}

]

pso_params = [    
    # Very high swarm size and iterations
 {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500}
]



# Initialize an empty DataFrame to store results
results = pd.DataFrame(columns=['ANN_Topology', 'PSO_Params', 'MAE_Train', 'MAE_Test'])

# Iterate over each ANN topology and PSO configuration
for ann_config in ann_topologies:
    nodes = ann_config['nodes']
    activations = [getattr(af, func) for func in ann_config['activations']]
    number_of_inputs = 8  # Input feature count

    for pso_config in pso_params:
        if validate_user_inputs(number_of_inputs, nodes, activations):
            ann = ANNBuilder().build(number_of_inputs, nodes, activations)
            
            # Extract PSO parameters
            alpha, beta, gamma, delta, epsilon, swarm_size, max_iterations = (
                pso_config['alpha'], pso_config['beta'], pso_config['gamma'],
                pso_config['delta'], pso_config['epsilon'], pso_config['swarm_size'], pso_config['max_iterations']
            )

            # Run PSO to optimize the ANN weights
            pso = PSO(swarm_size, number_of_inputs, nodes, activations, alpha, beta, gamma, delta, epsilon, max_iterations)
            pso.optimize()

            # Set ANN weights and biases with PSO's best solution
            ann.set_weights_and_biases(pso.global_best_position)

            # Forward pass on training and test data
            output_train = ann.forward(X_train.T)
            output_test = ann.forward(X_test.T)

            # Rescale predictions back to original scale
            output_train_original = min_max_scaler_y.inverse_transform(output_train.T)
            output_test_original = min_max_scaler_y.inverse_transform(output_test.T)

            # Calculate MAE for training and test sets
            mae_train = mean_absolute_error(min_max_scaler_y.inverse_transform(y_train), output_train_original)
            mae_test = mean_absolute_error(min_max_scaler_y.inverse_transform(y_test), output_test_original)

                        # Replace the append method with pd.concat
            results = pd.concat([results, pd.DataFrame([{
                'ANN_Topology': ann_config,
                'PSO_Params': pso_config,
                'MAE_Train': mae_train,
                'MAE_Test': mae_test
            }])], ignore_index=True)

            # Debug: Print MAE values for each combination
            print(f"ANN Config: {ann_config}")
            print(f"PSO Config: {pso_config}")
            print(f"MAE Train: {mae_train}")
            print(f"MAE Test: {mae_test}")
            print("-" * 40)

# Save results to CSV for analysis
# results.to_csv("experiment_results_2.csv", index=False)

print("Experiment completed. Results saved to 'experiment_results_2.csv'.")


ANN Config: {'nodes': [8, 9, 6, 1], 'activations': ['tanh', 'relu', 'sigmoid', 'linear_activation']}
PSO Config: {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500}
MAE Train: 3.890808325444187
MAE Test: 4.690556511463504
----------------------------------------
Experiment completed. Results saved to 'experiment_results_2.csv'.


C:\Users\Hp\AppData\Local\Temp\ipykernel_1420\4029806468.py:93: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([{


In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Assuming necessary classes and functions (like ActivationFunctions, ANNBuilder, PSO, etc.) are defined
# Load the dataset
data = pd.read_csv('concrete_data.csv')

# Separate features and target
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize the feature values
min_max_scaler_X = MinMaxScaler()
X_train = min_max_scaler_X.fit_transform(X_train)
X_test = min_max_scaler_X.transform(X_test)

# Normalize the target values separately
min_max_scaler_y = MinMaxScaler()
y_train = min_max_scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test = min_max_scaler_y.transform(y_test.reshape(-1, 1))

# Create an instance of ActivationFunctions
af = ActivationFunctions()

# Define ANN and PSO configurations
ann_topologies = [
    {'nodes': [8, 10, 5, 1], 'activations': ['sigmoid', 'relu', 'tanh', 'linear_activation']},
    {'nodes': [8, 5, 3, 1], 'activations': ['tanh', 'tanh', 'relu', 'linear_activation']},
    {'nodes': [8, 6, 4, 1], 'activations': ['relu', 'tanh', 'tanh', 'linear_activation']},
    {'nodes': [8, 9, 6, 1], 'activations': ['tanh', 'relu', 'sigmoid', 'linear_activation']},
    # Configuration 1: Adding an additional layer with 'relu' activation
    {'nodes': [8, 10, 7, 4, 1], 'activations': ['tanh', 'relu', 'sigmoid', 'relu', 'linear_activation']},

    # Configuration 2: A deeper network with more layers and varied activations
    {'nodes': [8, 12, 8, 5, 3, 1], 'activations': ['relu', 'relu', 'tanh', 'sigmoid', 'tanh', 'linear_activation']},

    # Configuration 3: Adding more layers for testing model performance with different activation combinations
    {'nodes': [8, 14, 10, 6, 2, 1], 'activations': ['tanh', 'relu', 'relu', 'sigmoid', 'tanh', 'linear_activation']},

    # Configuration 4: A highly complex model with multiple hidden layers and diverse activations
    {'nodes': [8, 15, 12, 9, 5, 1], 'activations': ['relu', 'sigmoid', 'relu', 'tanh', 'sigmoid', 'linear_activation']}

]

pso_params = [    
    # Very high swarm size and iterations
    {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500},
]



# Initialize an empty DataFrame to store results
results = pd.DataFrame(columns=['ANN_Topology', 'PSO_Params', 'MAE_Train', 'MAE_Test'])

# Iterate over each ANN topology and PSO configuration
for ann_config in ann_topologies:
    nodes = ann_config['nodes']
    activations = [getattr(af, func) for func in ann_config['activations']]
    number_of_inputs = 8  # Input feature count

    for pso_config in pso_params:
        if validate_user_inputs(number_of_inputs, nodes, activations):
            ann = ANNBuilder().build(number_of_inputs, nodes, activations)
            
            # Extract PSO parameters
            alpha, beta, gamma, delta, epsilon, swarm_size, max_iterations = (
                pso_config['alpha'], pso_config['beta'], pso_config['gamma'],
                pso_config['delta'], pso_config['epsilon'], pso_config['swarm_size'], pso_config['max_iterations']
            )

            # Run PSO to optimize the ANN weights
            pso = PSO(swarm_size, number_of_inputs, nodes, activations, alpha, beta, gamma, delta, epsilon, max_iterations)
            pso.optimize()

            # Set ANN weights and biases with PSO's best solution
            ann.set_weights_and_biases(pso.global_best_position)

            # Forward pass on training and test data
            output_train = ann.forward(X_train.T)
            output_test = ann.forward(X_test.T)

            # Rescale predictions back to original scale
            output_train_original = min_max_scaler_y.inverse_transform(output_train.T)
            output_test_original = min_max_scaler_y.inverse_transform(output_test.T)

            # Calculate MAE for training and test sets
            mae_train = mean_absolute_error(min_max_scaler_y.inverse_transform(y_train), output_train_original)
            mae_test = mean_absolute_error(min_max_scaler_y.inverse_transform(y_test), output_test_original)

                        # Replace the append method with pd.concat
            results = pd.concat([results, pd.DataFrame([{
                'ANN_Topology': ann_config,
                'PSO_Params': pso_config,
                'MAE_Train': mae_train,
                'MAE_Test': mae_test
            }])], ignore_index=True)

            # Debug: Print MAE values for each combination
            print(f"ANN Config: {ann_config}")
            print(f"PSO Config: {pso_config}")
            print(f"MAE Train: {mae_train}")
            print(f"MAE Test: {mae_test}")
            print("-" * 40)

# Save results to CSV for analysis
results.to_csv("experiment_results_2.csv", index=False)

print("Experiment completed. Results saved to 'experiment_results_2.csv'.")


C:\Users\Hp\AppData\Local\Temp\ipykernel_1420\44040901.py:97: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([{


ANN Config: {'nodes': [8, 10, 5, 1], 'activations': ['sigmoid', 'relu', 'tanh', 'linear_activation']}
PSO Config: {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500}
MAE Train: 4.440720555627268
MAE Test: 4.968374502222864
----------------------------------------
ANN Config: {'nodes': [8, 5, 3, 1], 'activations': ['tanh', 'tanh', 'relu', 'linear_activation']}
PSO Config: {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500}
MAE Train: 4.147714978496239
MAE Test: 4.555395137619724
----------------------------------------
ANN Config: {'nodes': [8, 6, 4, 1], 'activations': ['relu', 'tanh', 'tanh', 'linear_activation']}
PSO Config: {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500}
MAE Train: 4.185116799369296
MAE Test: 4.71649451873143
----------------------------------------
ANN Config: {'nodes': [8, 9, 6, 1], '

PermissionError: [Errno 13] Permission denied: 'experiment_results_2.csv'

In [23]:
#MOST OPTIMISED PARAMETERS
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Assuming necessary classes and functions (like ActivationFunctions, ANNBuilder, PSO, etc.) are defined
# Load the dataset
data = pd.read_csv('concrete_data.csv')

# Separate features and target
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize the feature values
min_max_scaler_X = MinMaxScaler()
X_train = min_max_scaler_X.fit_transform(X_train)
X_test = min_max_scaler_X.transform(X_test)

# Normalize the target values separately
min_max_scaler_y = MinMaxScaler()
y_train = min_max_scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test = min_max_scaler_y.transform(y_test.reshape(-1, 1))

# Create an instance of ActivationFunctions
af = ActivationFunctions()

# Define ANN and PSO configurations
ann_topologies = [
    # {'nodes': [8, 10, 5, 1], 'activations': ['sigmoid', 'relu', 'tanh', 'linear_activation']},
    # {'nodes': [8, 5, 3, 1], 'activations': ['tanh', 'tanh', 'relu', 'linear_activation']},
    # {'nodes': [8, 6, 4, 1], 'activations': ['relu', 'tanh', 'tanh', 'linear_activation']},
    # {'nodes': [8, 10, 5, 1], 'activations': ['sigmoid', 'relu', 'tanh', 'linear_activation']},
    # {'nodes': [8, 3, 2, 1], 'activations': ['relu', 'relu', 'sigmoid', 'linear_activation']},
    # {'nodes': [8, 7, 5, 1], 'activations': ['tanh', 'sigmoid', 'relu', 'linear_activation']},
    # {'nodes': [8, 4, 4, 1], 'activations': ['relu', 'sigmoid', 'sigmoid', 'linear_activation']},
    # {'nodes': [8, 6, 2, 1], 'activations': ['tanh', 'relu', 'tanh', 'linear_activation']},
    # {'nodes': [8, 5, 3, 1], 'activations': ['sigmoid', 'sigmoid', 'relu', 'linear_activation']},
    # {'nodes': [8, 8, 4, 1], 'activations': ['relu', 'tanh', 'sigmoid', 'linear_activation']},
    {'nodes': [8, 9, 6, 1], 'activations': ['tanh', 'relu', 'sigmoid', 'linear_activation']}

]

pso_params = [    
    # Very high swarm size and iterations
 {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500}
]



# Initialize an empty DataFrame to store results
results = pd.DataFrame(columns=['ANN_Topology', 'PSO_Params', 'MAE_Train', 'MAE_Test'])

# Iterate over each ANN topology and PSO configuration
for ann_config in ann_topologies:
    nodes = ann_config['nodes']
    activations = [getattr(af, func) for func in ann_config['activations']]
    number_of_inputs = 8  # Input feature count

    for pso_config in pso_params:
        if validate_user_inputs(number_of_inputs, nodes, activations):
            ann = ANNBuilder().build(number_of_inputs, nodes, activations)
            
            # Extract PSO parameters
            alpha, beta, gamma, delta, epsilon, swarm_size, max_iterations = (
                pso_config['alpha'], pso_config['beta'], pso_config['gamma'],
                pso_config['delta'], pso_config['epsilon'], pso_config['swarm_size'], pso_config['max_iterations']
            )

            # Run PSO to optimize the ANN weights
            pso = PSO(swarm_size, number_of_inputs, nodes, activations, alpha, beta, gamma, delta, epsilon, max_iterations)
            pso.optimize()

            # Set ANN weights and biases with PSO's best solution
            ann.set_weights_and_biases(pso.global_best_position)

            # Forward pass on training and test data
            output_train = ann.forward(X_train.T)
            output_test = ann.forward(X_test.T)

            # Rescale predictions back to original scale
            output_train_original = min_max_scaler_y.inverse_transform(output_train.T)
            output_test_original = min_max_scaler_y.inverse_transform(output_test.T)

            # Calculate MAE for training and test sets
            mae_train = mean_absolute_error(min_max_scaler_y.inverse_transform(y_train), output_train_original)
            mae_test = mean_absolute_error(min_max_scaler_y.inverse_transform(y_test), output_test_original)

                        # Replace the append method with pd.concat
            results = pd.concat([results, pd.DataFrame([{
                'ANN_Topology': ann_config,
                'PSO_Params': pso_config,
                'MAE_Train': mae_train,
                'MAE_Test': mae_test
            }])], ignore_index=True)

            # Debug: Print MAE values for each combination
            print(f"ANN Config: {ann_config}")
            print(f"PSO Config: {pso_config}")
            print(f"MAE Train: {mae_train}")
            print(f"MAE Test: {mae_test}")
            print("-" * 40)

# Save results to CSV for analysis
# results.to_csv("experiment_results_2.csv", index=False)

print("Experiment completed. Results saved to 'experiment_results_2.csv'.")


ANN Config: {'nodes': [8, 9, 6, 1], 'activations': ['tanh', 'relu', 'sigmoid', 'linear_activation']}
PSO Config: {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500}
MAE Train: 4.471834054153647
MAE Test: 5.268755257142431
----------------------------------------
Experiment completed. Results saved to 'experiment_results_2.csv'.


C:\Users\Hp\AppData\Local\Temp\ipykernel_1420\1028385870.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([{


In [24]:
#RUNNING THE ANN 10 TIMES AND TAKING THE AVG OF MAE

import numpy as np

# Putting It All Together
if __name__ == "__main__":
    # Load dataset
    data = pd.read_csv('concrete_data.csv')
    
    # Separate features and target
    X = data.iloc[:, :-1].values  # All rows, all columns except the last one
    y = data.iloc[:, -1].values   # All rows, only the last column (compressive strength)

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Normalize the feature values
    min_max_scaler_X = MinMaxScaler()
    X_train = min_max_scaler_X.fit_transform(X_train)
    X_test = min_max_scaler_X.transform(X_test)
    
    # Normalize the target values separately
    min_max_scaler_y = MinMaxScaler()
    y_train = min_max_scaler_y.fit_transform(y_train.reshape(-1, 1))
    y_test = min_max_scaler_y.transform(y_test.reshape(-1, 1))

    # Create an instance of ActivationFunctions
    af = ActivationFunctions()

    # User inputs for the network
    number_of_inputs = 8  # Number of input features
    number_of_nodes = [8, 9, 6, 1]  # Adding more neurons per layer
    activation_functions = [af.tanh, af.relu, af.sigmoid, af.linear_activation]
    #{'nodes': [8, 9, 6, 1], 'activations': ['tanh', 'relu', 'sigmoid', 'linear_activation']}

    # PSO parameters
    #  {'alpha': 0.5, 'beta': 1.0, 'gamma': 1.0, 'delta': 1, 'epsilon': 1.2, 'swarm_size': 200, 'max_iterations': 500}

    swarm_size = 200
    alpha = 0.5
    beta = 1.0
    gamma = 1.0
    delta = 1.0
    epsilon = 1.2
    max_iterations = 500

    # Lists to store MAE results
    mae_train_results = []
    mae_test_results = []

    # Run the code 10 times
    for _ in range(10):
        # Run PSO to optimize the ANN
        pso = PSO(swarm_size, number_of_inputs, number_of_nodes, activation_functions, alpha, beta, gamma, delta, epsilon, max_iterations)
        pso.optimize()

        # Create the network with optimized weights
        ann = ANNBuilder().build(number_of_inputs, number_of_nodes, activation_functions)
        ann.set_weights_and_biases(pso.global_best_position)

        # Perform a forward pass with training data
        input_data = X_train.T  # Transpose to match input shape
        output = ann.forward(input_data)

        # Inverse transform the predictions to the original scale
        output_original_scale = min_max_scaler_y.inverse_transform(output.T)

        # Calculate Mean Absolute Error on training data in original scale
        mae_train_original = mae(min_max_scaler_y.inverse_transform(y_train), output_original_scale)
        mae_train_results.append(mae_train_original)

        # Perform a forward pass with test data
        test_output = ann.forward(X_test.T)

        # Inverse transform the predictions to the original scale
        test_output_original_scale = min_max_scaler_y.inverse_transform(test_output.T)

        # Calculate Mean Absolute Error on test data in original scale
        mae_test_original = mae(min_max_scaler_y.inverse_transform(y_test), test_output_original_scale)
        mae_test_results.append(mae_test_original)

    # Calculate average and standard deviation of MAE for training and test data
    avg_mae_train = np.mean(mae_train_results)
    std_mae_train = np.std(mae_train_results)
    avg_mae_test = np.mean(mae_test_results)
    std_mae_test = np.std(mae_test_results)

    print(f"Average MAE on training data (original scale): {avg_mae_train:.2f} ± {std_mae_train:.2f}")
    print(f"Average MAE on test data (original scale): {avg_mae_test:.2f} ± {std_mae_test:.2f}")


Average MAE on training data (original scale): 4.17 ± 0.24
Average MAE on test data (original scale): 4.80 ± 0.22
